In [18]:

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import col, round, to_date, year, month, sum, avg

conf = (
     SparkConf()
    .setAppName('Simple_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)

In [19]:
# Пример данных
data = [(i, f"Name_{i % 5}") for i in range(100)]
df = spark.createDataFrame(data, ["id", "name"])

In [3]:
df.show(5)

+---+------+
| id|  name|
+---+------+
|  0|Name_0|
|  1|Name_1|
|  2|Name_2|
|  3|Name_3|
|  4|Name_4|
+---+------+
only showing top 5 rows



In [20]:
# Проверяем начальное количество разделов
initial_partitions = df.rdd.getNumPartitions()
initial_partitions

4

In [5]:
# Применяем repartition для увеличения количества разделов до 15
df_repartitioned = df.repartition(15)

# проверим партиции
df_repartitioned.rdd.getNumPartitions()

15

In [6]:
# Пример использования для улучшения выполнения join
other_data = [(i, f"OtherName_{i % 5}") for i in range(100)]
other_df = spark.createDataFrame(other_data, ["id", "other_name"])

In [7]:
repartitioned_df = df_repartitioned.repartition(10, "id")
joined_df = repartitioned_df.join(other_df, "id")

In [8]:
joined_df.show(5)

+---+------+-----------+
| id|  name| other_name|
+---+------+-----------+
| 26|Name_1|OtherName_1|
| 29|Name_4|OtherName_4|
| 65|Name_0|OtherName_0|
| 19|Name_4|OtherName_4|
| 54|Name_4|OtherName_4|
+---+------+-----------+
only showing top 5 rows



In [9]:
joined_df.rdd.getNumPartitions()

200

#### __partitionBy__

__partitionBy__ - это метод, используемый при записи данных в файлы (например, в форматах Parquet, ORC, etc.) для указания того, как данные должны быть разделены на партиции. Он определяет логику разделения данных на основе значений одного или нескольких столбцов

In [12]:
# Если читаем с CSV какого - нибудь, то также можно указать, сколько партиций должно быть
df_sales = spark.read.csv("sales.csv", header=True, inferSchema=True, numPartitions=10)

TypeError: DataFrameReader.csv() got an unexpected keyword argument 'numPartitions'

#### coalesce

__coalesce__ — это операция в Apache Spark, которая используется для уменьшения количества партиций (разделов) в DataFrame или RDD. В отличие от repartition, которая может как увеличивать, так и уменьшать количество партиций и всегда включает shuffle (перераспределение данных между узлами), coalesce оптимизирована для уменьшения количества партиций без shuffle, если это возможно.

In [22]:
# Проверяем начальное количество партиций
initial_partitions = df.rdd.getNumPartitions()
initial_partitions

4

In [26]:
# Применяем repartition для увеличения количества партиций до 10 (для демонстрации)
df_repartitioned = df.repartition(10)
df_repartitioned.rdd.getNumPartitions()

10

In [27]:
df_coalesced = df_repartitioned.coalesce(3)
df_coalesced.rdd.getNumPartitions()

3